In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


from darian.learn.isolation_forest import IsolationForest


In [2]:
#read the input data

# Data preparation
trips_da ta =  pd.read_csv('G:/NYCCitiBikeSystem_AnomalyDetection/data/trips_data_Jan_2022.csv')
trips_data.head()

C:\ProgramData\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3526: DtypeWarning: Columns (5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,start_time,start_date,end_time,end_date,day_of_week,hour,trip_duration
0,55262E4365A955A2,classic_bike,2022-01-18T08:23:52Z,2022-01-18T08:28:18Z,Boerum Pl\t& Pacific St,4488.09,Clinton St & Joralemon St,4605.04,40.688489,-73.991160,40.692395,-73.993379,member,08:23:52,2022-01-18,08:28:18,2022-01-18,Tuesday,8,4.433333
1,D272F1B15D841EC0,classic_bike,2022-01-21T09:03:22Z,2022-01-21T09:05:44Z,E 12 St & Ave C,5616.08,E 10 St & Avenue A,5659.05,40.727243,-73.976831,40.727408,-73.981420,member,09:03:22,2022-01-21,09:05:44,2022-01-21,Friday,9,2.366667
2,D1FCEF55EB4A807F,classic_bike,2022-01-22T14:28:32Z,2022-01-22T14:53:18Z,W 21 St & 6 Ave,6140.05,W 44 St & 11 Ave,6756.05,40.741740,-73.994156,40.762009,-73.996975,member,14:28:32,2022-01-22,14:53:18,2022-01-22,Saturday,14,24.766667
3,E9CBDC6A0162C068,electric_bike,2022-01-19T14:49:47Z,2022-01-19T14:54:02Z,38 St & 30 Ave,6850.01,Crescent St & 30 Ave,6958.06,40.764175,-73.915840,40.768692,-73.924957,member,14:49:47,2022-01-19,14:54:02,2022-01-19,Wednesday,14,4.250000
4,2177A5B57326CE9B,electric_bike,2022-01-16T14:36:06Z,2022-01-16T14:44:06Z,Pacific St & Nevins St,4362.04,Clinton St & Tillary St,4748.07,40.685376,-73.983021,40.696233,-73.991421,member,14:36:06,2022-01-16,14:44:06,2022-01-16,Sunday,14,8.000000


In [3]:
trips_data.dtypes

ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
start_time             object
start_date             object
end_time               object
end_date               object
day_of_week            object
hour                    int64
trip_duration         float64
dtype: object

In [4]:
trips_data['rideable_type'] = trips_data['rideable_type'].astype('category')
trips_data['member_casual'] = trips_data['member_casual'].astype('category')
trips_data['day_of_week'] = trips_data['day_of_week'].astype('category')
trips_data['start_station_id'] = trips_data['start_station_id'].astype('category')
trips_data['end_station_id'] = trips_data['end_station_id'].astype('category')
trips_data['hour'] = trips_data['hour'].astype('category')

In [5]:
trips_data.dtypes

ride_id                 object
rideable_type         category
started_at              object
ended_at                object
start_station_name      object
start_station_id      category
end_station_name        object
end_station_id        category
start_lat              float64
start_lng              float64
end_lat                float64
end_lng                float64
member_casual         category
start_time              object
start_date              object
end_time                object
end_date                object
day_of_week           category
hour                  category
trip_duration          float64
dtype: object

In [6]:
trips_data.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,start_time,start_date,end_time,end_date,day_of_week,hour,trip_duration
0,55262E4365A955A2,classic_bike,2022-01-18T08:23:52Z,2022-01-18T08:28:18Z,Boerum Pl\t& Pacific St,4488.09,Clinton St & Joralemon St,4605.04,40.688489,-73.991160,40.692395,-73.993379,member,08:23:52,2022-01-18,08:28:18,2022-01-18,Tuesday,8,4.433333
1,D272F1B15D841EC0,classic_bike,2022-01-21T09:03:22Z,2022-01-21T09:05:44Z,E 12 St & Ave C,5616.08,E 10 St & Avenue A,5659.05,40.727243,-73.976831,40.727408,-73.981420,member,09:03:22,2022-01-21,09:05:44,2022-01-21,Friday,9,2.366667
2,D1FCEF55EB4A807F,classic_bike,2022-01-22T14:28:32Z,2022-01-22T14:53:18Z,W 21 St & 6 Ave,6140.05,W 44 St & 11 Ave,6756.05,40.741740,-73.994156,40.762009,-73.996975,member,14:28:32,2022-01-22,14:53:18,2022-01-22,Saturday,14,24.766667
3,E9CBDC6A0162C068,electric_bike,2022-01-19T14:49:47Z,2022-01-19T14:54:02Z,38 St & 30 Ave,6850.01,Crescent St & 30 Ave,6958.06,40.764175,-73.915840,40.768692,-73.924957,member,14:49:47,2022-01-19,14:54:02,2022-01-19,Wednesday,14,4.250000
4,2177A5B57326CE9B,electric_bike,2022-01-16T14:36:06Z,2022-01-16T14:44:06Z,Pacific St & Nevins St,4362.04,Clinton St & Tillary St,4748.07,40.685376,-73.983021,40.696233,-73.991421,member,14:36:06,2022-01-16,14:44:06,2022-01-16,Sunday,14,8.000000


In [7]:
# select variables
features = ['rideable_type','start_station_id','end_station_id','member_casual','day_of_week','trip_duration','hour']

training_data = trips_data[features]


model_1 = IsolationForest(n_estimators=100, max_samples=256, bootstrap = True, n_jobs=11)
model_1.fit(training_data)


In [8]:
trips_data['anomaly_scores'] = model_1.score_samples(training_data)

In [9]:
trips_data.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,end_lng,member_casual,start_time,start_date,end_time,end_date,day_of_week,hour,trip_duration,anomaly_scores
0,55262E4365A955A2,classic_bike,2022-01-18T08:23:52Z,2022-01-18T08:28:18Z,Boerum Pl\t& Pacific St,4488.09,Clinton St & Joralemon St,4605.04,40.688489,-73.991160,...,-73.993379,member,08:23:52,2022-01-18,08:28:18,2022-01-18,Tuesday,8,4.433333,0.469605
1,D272F1B15D841EC0,classic_bike,2022-01-21T09:03:22Z,2022-01-21T09:05:44Z,E 12 St & Ave C,5616.08,E 10 St & Avenue A,5659.05,40.727243,-73.976831,...,-73.981420,member,09:03:22,2022-01-21,09:05:44,2022-01-21,Friday,9,2.366667,0.524670
2,D1FCEF55EB4A807F,classic_bike,2022-01-22T14:28:32Z,2022-01-22T14:53:18Z,W 21 St & 6 Ave,6140.05,W 44 St & 11 Ave,6756.05,40.741740,-73.994156,...,-73.996975,member,14:28:32,2022-01-22,14:53:18,2022-01-22,Saturday,14,24.766667,0.496584
3,E9CBDC6A0162C068,electric_bike,2022-01-19T14:49:47Z,2022-01-19T14:54:02Z,38 St & 30 Ave,6850.01,Crescent St & 30 Ave,6958.06,40.764175,-73.915840,...,-73.924957,member,14:49:47,2022-01-19,14:54:02,2022-01-19,Wednesday,14,4.250000,0.503166
4,2177A5B57326CE9B,electric_bike,2022-01-16T14:36:06Z,2022-01-16T14:44:06Z,Pacific St & Nevins St,4362.04,Clinton St & Tillary St,4748.07,40.685376,-73.983021,...,-73.991421,member,14:36:06,2022-01-16,14:44:06,2022-01-16,Sunday,14,8.000000,0.498332


In [10]:
max_anomaly_score = trips_data['anomaly_scores'].max()
print("Maximum Anomaly Score:", max_anomaly_score)

Maximum Anomaly Score: 0.6880105682488632


In [11]:
min_anomaly_score = trips_data['anomaly_scores'].min()
print("Minimum Anomaly Score:", min_anomaly_score)

Minimum Anomaly Score: 0.4057578007813397


In [12]:
# Export the DataFrame to a CSV file
Jan_df = trips_data.to_csv('G:/NYCCitiBikeSystem_AnomalyDetection/data/Jan_df.csv', index=False)


In [13]:
print(Jan_df)

None


In [21]:
df1 = trips_data[trips_data['anomaly_scores'] > 0.4]
df1

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,end_lng,member_casual,start_time,start_date,end_time,end_date,day_of_week,hour,trip_duration,anomaly_scores
0,55262E4365A955A2,classic_bike,2022-01-18T08:23:52Z,2022-01-18T08:28:18Z,Boerum Pl\t& Pacific St,4488.09,Clinton St & Joralemon St,4605.04,40.688489,-73.991160,...,-73.993379,member,08:23:52,2022-01-18,08:28:18,2022-01-18,Tuesday,8,4.433333,0.469605
1,D272F1B15D841EC0,classic_bike,2022-01-21T09:03:22Z,2022-01-21T09:05:44Z,E 12 St & Ave C,5616.08,E 10 St & Avenue A,5659.05,40.727243,-73.976831,...,-73.981420,member,09:03:22,2022-01-21,09:05:44,2022-01-21,Friday,9,2.366667,0.524670
2,D1FCEF55EB4A807F,classic_bike,2022-01-22T14:28:32Z,2022-01-22T14:53:18Z,W 21 St & 6 Ave,6140.05,W 44 St & 11 Ave,6756.05,40.741740,-73.994156,...,-73.996975,member,14:28:32,2022-01-22,14:53:18,2022-01-22,Saturday,14,24.766667,0.496584
3,E9CBDC6A0162C068,electric_bike,2022-01-19T14:49:47Z,2022-01-19T14:54:02Z,38 St & 30 Ave,6850.01,Crescent St & 30 Ave,6958.06,40.764175,-73.915840,...,-73.924957,member,14:49:47,2022-01-19,14:54:02,2022-01-19,Wednesday,14,4.250000,0.503166
4,2177A5B57326CE9B,electric_bike,2022-01-16T14:36:06Z,2022-01-16T14:44:06Z,Pacific St & Nevins St,4362.04,Clinton St & Tillary St,4748.07,40.685376,-73.983021,...,-73.991421,member,14:36:06,2022-01-16,14:44:06,2022-01-16,Sunday,14,8.000000,0.498332
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1012198,3A8436E896501EB6,classic_bike,2022-01-09T09:40:49Z,2022-01-09T09:58:55Z,27 St & Hunter St,6310.06,Vernon Blvd & 50 Ave,6170.02,40.748500,-73.941275,...,-73.954117,casual,09:40:49,2022-01-09,09:58:55,2022-01-09,Sunday,9,18.100000,0.569400
1012199,31EE3AF319AE47D2,classic_bike,2022-01-26T16:11:27Z,2022-01-26T16:17:20Z,Monroe St & Bedford Ave,4368.05,Putnam Ave & Throop Ave,4392.04,40.685144,-73.953809,...,-73.941110,casual,16:11:27,2022-01-26,16:17:20,2022-01-26,Wednesday,16,5.883333,0.534820
1012200,C27D809EC458BE2A,electric_bike,2022-01-20T11:39:41Z,2022-01-20T11:44:46Z,E 118 St & 1 Ave,7596.1,Lexington Ave & E 111 St,7567.06,40.797470,-73.935040,...,-73.944123,member,11:39:41,2022-01-20,11:44:46,2022-01-20,Thursday,11,5.083333,0.509110
1012201,4968572973DEDCB2,electric_bike,2022-01-18T21:17:11Z,2022-01-18T21:25:57Z,Allen St & Hester St,5342.1,1 Ave & E 18 St,5854.09,40.716059,-73.991908,...,-73.980544,member,21:17:11,2022-01-18,21:25:57,2022-01-18,Tuesday,21,8.766667,0.475930


In [24]:
trips_data[(trips_data['anomaly_scores'] > 0.4) & (trips_data['anomaly_scores'] < 0.5)]

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,end_lng,member_casual,start_time,start_date,end_time,end_date,day_of_week,hour,trip_duration,anomaly_scores
0,55262E4365A955A2,classic_bike,2022-01-18T08:23:52Z,2022-01-18T08:28:18Z,Boerum Pl\t& Pacific St,4488.09,Clinton St & Joralemon St,4605.04,40.688489,-73.991160,...,-73.993379,member,08:23:52,2022-01-18,08:28:18,2022-01-18,Tuesday,8,4.433333,0.469605
2,D1FCEF55EB4A807F,classic_bike,2022-01-22T14:28:32Z,2022-01-22T14:53:18Z,W 21 St & 6 Ave,6140.05,W 44 St & 11 Ave,6756.05,40.741740,-73.994156,...,-73.996975,member,14:28:32,2022-01-22,14:53:18,2022-01-22,Saturday,14,24.766667,0.496584
4,2177A5B57326CE9B,electric_bike,2022-01-16T14:36:06Z,2022-01-16T14:44:06Z,Pacific St & Nevins St,4362.04,Clinton St & Tillary St,4748.07,40.685376,-73.983021,...,-73.991421,member,14:36:06,2022-01-16,14:44:06,2022-01-16,Sunday,14,8.000000,0.498332
6,5E4425CCF4C92012,electric_bike,2022-01-28T17:12:46Z,2022-01-28T17:27:03Z,Kosciuszko St & Nostrand Ave,4519.04,Eastern Pkwy & Washington Ave,3928.08,40.690725,-73.951335,...,-73.963115,member,17:12:46,2022-01-28,17:27:03,2022-01-28,Friday,17,14.283333,0.485849
7,B10B3DB76684E8F4,classic_bike,2022-01-04T22:12:48Z,2022-01-04T22:18:27Z,W 21 St & 6 Ave,6140.05,E 20 St & 2 Ave,5971.08,40.741740,-73.994156,...,-73.982050,member,22:12:48,2022-01-04,22:18:27,2022-01-04,Tuesday,22,5.650000,0.462605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1012187,A4EBFD2F8C198689,electric_bike,2022-01-11T18:16:48Z,2022-01-11T18:19:52Z,E 7 St & Avenue A,5626.07,E 5 St & Cooper Sq,5712.12,40.726218,-73.983799,...,-73.990993,member,18:16:48,2022-01-11,18:19:52,2022-01-11,Tuesday,18,3.066667,0.487053
1012189,C561E2CB13941E51,electric_bike,2022-01-09T09:49:46Z,2022-01-09T10:06:04Z,Allen St & Hester St,5342.1,E 32 St & Park Ave,6280.12,40.716059,-73.991908,...,-73.981948,member,09:49:46,2022-01-09,10:06:04,2022-01-09,Sunday,9,16.300000,0.499385
1012190,8368AD6C9F8C2A94,classic_bike,2022-01-23T11:01:02Z,2022-01-23T11:09:28Z,E 23 St & 1 Ave,5929.01,E 32 St & Park Ave,6280.12,40.736502,-73.978095,...,-73.981948,member,11:01:02,2022-01-23,11:09:28,2022-01-23,Sunday,11,8.433333,0.456923
1012194,9BD4C2B27FFC072D,classic_bike,2022-01-18T17:03:10Z,2022-01-18T17:09:22Z,E 9 St & 5 Ave,5872.1,MacDougal St & Prince St,5687.04,40.732759,-73.995817,...,-74.002971,member,17:03:10,2022-01-18,17:09:22,2022-01-18,Tuesday,17,6.200000,0.492479


In [25]:
df = trips_data[trips_data['anomaly_scores'] < 0.45]
df

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,end_lng,member_casual,start_time,start_date,end_time,end_date,day_of_week,hour,trip_duration,anomaly_scores
12,31028A942DB7C631,classic_bike,2022-01-14T12:03:50Z,2022-01-14T12:18:52Z,W 21 St & 6 Ave,6140.05,E 45 St & 3 Ave,6464.08,40.741740,-73.994156,...,-73.972826,member,12:03:50,2022-01-14,12:18:52,2022-01-14,Friday,12,15.033333,0.448571
71,582C4FB96E010416,classic_bike,2022-01-18T08:45:24Z,2022-01-18T08:54:11Z,E 88 St & 1 Ave,7235.13,2 Ave & E 72 St,6925.09,40.778301,-73.948813,...,-73.958408,member,08:45:24,2022-01-18,08:54:11,2022-01-18,Tuesday,8,8.783333,0.440881
92,ABC1032FFBD0AB29,electric_bike,2022-01-02T15:33:09Z,2022-01-02T15:37:43Z,W 21 St & 6 Ave,6140.05,W 20 St & 8 Ave,6224.05,40.741740,-73.994156,...,-74.000040,member,15:33:09,2022-01-02,15:37:43,2022-01-02,Sunday,15,4.566667,0.447714
165,6084F551460A2C24,classic_bike,2022-01-21T12:25:45Z,2022-01-21T12:30:12Z,12 Ave & W 40 St,6765.01,W 44 St & 11 Ave,6756.05,40.760875,-74.002777,...,-73.996975,member,12:25:45,2022-01-21,12:30:12,2022-01-21,Friday,12,4.450000,0.438850
306,898F45CDDE256C38,classic_bike,2022-01-18T12:42:52Z,2022-01-18T12:49:25Z,W 59 St & 10 Ave,7023.04,W 44 St & 11 Ave,6756.05,40.770513,-73.988038,...,-73.996975,member,12:42:52,2022-01-18,12:49:25,2022-01-18,Tuesday,12,6.550000,0.436698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1010813,AEE384C27F692CE2,classic_bike,2022-01-17T12:22:19Z,2022-01-17T12:30:12Z,E 7 St & Avenue A,5626.07,E 25 St & 1 Ave,6004.07,40.726218,-73.983799,...,-73.977387,member,12:22:19,2022-01-17,12:30:12,2022-01-17,Monday,12,7.883333,0.440714
1011742,16A464C19F496280,classic_bike,2022-01-12T19:16:37Z,2022-01-12T19:26:40Z,Madison Av & E 51 St,6659.09,E 74 St & 1 Ave,6953.08,40.758630,-73.975130,...,-73.954823,member,19:16:37,2022-01-12,19:26:40,2022-01-12,Wednesday,19,10.050000,0.449103
1011847,A81DB9D46621B16E,classic_bike,2022-01-10T15:16:34Z,2022-01-10T15:21:16Z,E 23 St & 1 Ave,5929.01,1 Ave & E 18 St,5854.09,40.736502,-73.978095,...,-73.980544,member,15:16:34,2022-01-10,15:21:16,2022-01-10,Monday,15,4.700000,0.447376
1012027,E8A1C6E693E5BA78,electric_bike,2022-01-06T19:00:30Z,2022-01-06T19:09:43Z,Allen St & Hester St,5342.1,S 5 Pl & S 5 St,5125.03,40.716059,-73.991908,...,-73.960876,member,19:00:30,2022-01-06,19:09:43,2022-01-06,Thursday,19,9.216667,0.447211


In [22]:
# Export the DataFrame to a CSV file
df1 = df1.to_csv('G:/NYCCitiBikeSystem_AnomalyDetection/data/df.csv', index=False)

In [16]:
trips_data[trips_data['anomaly_scores'] > 0.6]

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,end_lng,member_casual,start_time,start_date,end_time,end_date,day_of_week,hour,trip_duration,anomaly_scores
14,E4DDFD0AF8171296,electric_bike,2022-01-24T16:33:02Z,2022-01-24T19:29:11Z,St. Nicholas Terrace & Convent Ave,7937.15,Grand Concourse & E 164 St,8057.02,40.821294,-73.947868,...,-73.921461,member,16:33:02,2022-01-24,19:29:11,2022-01-24,Monday,16,176.150000,0.627859
60,B080C2C425EBFF54,electric_bike,2022-01-13T08:18:25Z,2022-01-13T09:05:35Z,Avenue D & E 3 St,5436.09,E 106 St & Madison Ave,7528.31,40.720828,-73.977932,...,-73.949450,member,08:18:25,2022-01-13,09:05:35,2022-01-13,Thursday,8,47.166667,0.613698
91,B379B77AEB677F2A,electric_bike,2022-01-01T15:48:06Z,2022-01-01T16:44:21Z,Central Park W & W 91 St,7453.01,5 Ave & E 87 St,7323.09,40.788665,-73.966801,...,-73.959875,casual,15:48:06,2022-01-01,16:44:21,2022-01-01,Saturday,15,56.250000,0.637274
94,8D5737E07A2448BA,electric_bike,2022-01-20T00:19:16Z,2022-01-20T01:09:40Z,Kingston Ave & Herkimer St,4205.06,Brooklyn Bridge Park - Pier 2,4756.04,40.678907,-73.941428,...,-73.997178,member,00:19:16,2022-01-20,01:09:40,2022-01-20,Thursday,0,50.400000,0.618188
102,A70672024C1294DE,electric_bike,2022-01-02T06:50:44Z,2022-01-02T08:19:37Z,Cliff St & Fulton St,5065.14,John St & William St,5065.04,40.708380,-74.004950,...,-74.007222,member,06:50:44,2022-01-02,08:19:37,2022-01-02,Sunday,6,88.883333,0.613420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1011639,F9DAC5E0DCDD7342,electric_bike,2022-01-13T22:36:27Z,2022-01-14T08:10:49Z,W 42 St & 8 Ave,6602.05,8 Ave & W 33 St,6450.12,40.757570,-73.990985,...,-73.993934,casual,22:36:27,2022-01-13,08:10:49,2022-01-14,Thursday,22,574.366667,0.660519
1011708,94C59AD118CE24AF,classic_bike,2022-01-02T09:18:05Z,2022-01-02T10:06:47Z,27 St & Hunter St,6310.06,Front St & Jay St,4895.03,40.748500,-73.941275,...,-73.986842,casual,09:18:05,2022-01-02,10:06:47,2022-01-02,Sunday,9,48.700000,0.656077
1011762,D0BD833E856EB132,electric_bike,2022-01-01T00:13:36Z,2022-01-01T00:41:08Z,Bushwick Ave & Stagg St,5140.06,Front St & Jay St,4895.03,40.709897,-73.940080,...,-73.986842,casual,00:13:36,2022-01-01,00:41:08,2022-01-01,Saturday,0,27.533333,0.603326
1011789,CBFCD552DAFB9CF9,electric_bike,2022-01-04T16:33:16Z,2022-01-04T17:24:18Z,E 75 St & 3 Ave,6991.12,W 148 St & Amsterdam Ave,8033.09,40.771129,-73.957723,...,-73.945909,casual,16:33:16,2022-01-04,17:24:18,2022-01-04,Tuesday,16,51.033333,0.642517


In [17]:
trips_data[trips_data['anomaly_scores'] > 0.65]

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,end_lng,member_casual,start_time,start_date,end_time,end_date,day_of_week,hour,trip_duration,anomaly_scores
873,91F6CD42067F8EA5,classic_bike,2022-01-08T16:11:35Z,2022-01-08T17:32:28Z,Vernon Blvd & 50 Ave,6170.02,Queens Plaza North & Crescent St,6429.01,40.742327,-73.954117,...,-73.940737,casual,16:11:35,2022-01-08,17:32:28,2022-01-08,Saturday,16,80.883333,0.657765
1055,1EB168A9A4CCC2B4,classic_bike,2022-01-02T14:06:51Z,2022-01-02T15:07:03Z,48 St & Broadway,6629.06,9 St & 44 Rd,6361.03,40.755763,-73.912715,...,-73.952100,casual,14:06:51,2022-01-02,15:07:03,2022-01-02,Sunday,14,60.200000,0.655136
1267,76A67AD7211539C9,electric_bike,2022-01-02T22:01:39Z,2022-01-02T23:24:59Z,Broadway\t& W 48 St,6708.04,Clinton St & Tillary St,4748.07,40.760177,-73.984868,...,-73.991421,casual,22:01:39,2022-01-02,23:24:59,2022-01-02,Sunday,22,83.333333,0.665501
1496,9E07E94B9D828166,electric_bike,2022-01-02T18:51:13Z,2022-01-02T19:38:23Z,W 74 St & Columbus Ave,7230.1,E 116 St & 2 Ave,7563.06,40.778567,-73.977550,...,-73.937261,casual,18:51:13,2022-01-02,19:38:23,2022-01-02,Sunday,18,47.166667,0.651540
1967,F1F48070E2ABF918,electric_bike,2022-01-02T22:01:38Z,2022-01-02T23:25:00Z,Broadway\t& W 48 St,6708.04,Clinton St & Tillary St,4748.07,40.760177,-73.984868,...,-73.991421,casual,22:01:38,2022-01-02,23:25:00,2022-01-02,Sunday,22,83.366667,0.665501
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1010768,421BF85804C9D2C9,classic_bike,2022-01-01T13:35:42Z,2022-01-01T14:59:58Z,5 Ave & E 87 St,7323.09,5 Ave & E 72 St,7100.07,40.782323,-73.959875,...,-73.966853,casual,13:35:42,2022-01-01,14:59:58,2022-01-01,Saturday,13,84.266667,0.672169
1010997,0E88F0CB00D6446A,electric_bike,2022-01-08T16:22:43Z,2022-01-08T17:29:02Z,Pike St & E Broadway,5270.05,Washington St & Gansevoort St,6039.06,40.714067,-73.992939,...,-74.008119,casual,16:22:43,2022-01-08,17:29:02,2022-01-08,Saturday,16,66.316667,0.656401
1011423,1F936B6D7F7453FA,classic_bike,2022-01-04T21:57:32Z,2022-01-05T13:56:05Z,Broadway & Roebling St,5125.07,N 6 St & Bedford Ave,5379.1,40.709248,-73.960631,...,-73.958509,casual,21:57:32,2022-01-04,13:56:05,2022-01-05,Tuesday,21,958.550000,0.679189
1011639,F9DAC5E0DCDD7342,electric_bike,2022-01-13T22:36:27Z,2022-01-14T08:10:49Z,W 42 St & 8 Ave,6602.05,8 Ave & W 33 St,6450.12,40.757570,-73.990985,...,-73.993934,casual,22:36:27,2022-01-13,08:10:49,2022-01-14,Thursday,22,574.366667,0.660519


In [18]:
trips_data[trips_data['anomaly_scores'] > 0.66]

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,end_lng,member_casual,start_time,start_date,end_time,end_date,day_of_week,hour,trip_duration,anomaly_scores
1267,76A67AD7211539C9,electric_bike,2022-01-02T22:01:39Z,2022-01-02T23:24:59Z,Broadway\t& W 48 St,6708.04,Clinton St & Tillary St,4748.07,40.760177,-73.984868,...,-73.991421,casual,22:01:39,2022-01-02,23:24:59,2022-01-02,Sunday,22,83.333333,0.665501
1967,F1F48070E2ABF918,electric_bike,2022-01-02T22:01:38Z,2022-01-02T23:25:00Z,Broadway\t& W 48 St,6708.04,Clinton St & Tillary St,4748.07,40.760177,-73.984868,...,-73.991421,casual,22:01:38,2022-01-02,23:25:00,2022-01-02,Sunday,22,83.366667,0.665501
3146,257DA87455945ED2,classic_bike,2022-01-13T10:10:59Z,2022-01-13T11:27:03Z,W 49 St & 8 Ave,6747.06,W 50 St & 9 Ave,6854.05,40.762272,-73.987882,...,-73.989180,casual,10:10:59,2022-01-13,11:27:03,2022-01-13,Thursday,10,76.066667,0.662466
3382,E97959FB9DFF64B6,classic_bike,2022-01-10T13:17:54Z,2022-01-10T14:30:38Z,Broadway & Moylan Pl,7823.03,W 37 St & Broadway,6441.06,40.814326,-73.959025,...,-73.987706,casual,13:17:54,2022-01-10,14:30:38,2022-01-10,Monday,13,72.733333,0.667487
4461,9E2F53879E473C71,classic_bike,2022-01-25T12:19:15Z,2022-01-25T14:02:30Z,5 Ave & E 72 St,7100.07,5 Ave & E 78 St,7161.08,40.772828,-73.966853,...,-73.964274,casual,12:19:15,2022-01-25,14:02:30,2022-01-25,Tuesday,12,103.250000,0.669971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1009456,101A72E7E426BBDC,classic_bike,2022-01-02T15:09:43Z,2022-01-02T16:18:48Z,5 Ave & E 87 St,7323.09,5 Ave & E 72 St,7100.07,40.782323,-73.959875,...,-73.966853,casual,15:09:43,2022-01-02,16:18:48,2022-01-02,Sunday,15,69.083333,0.666355
1009464,04A4B63FA9BF2E01,classic_bike,2022-01-01T13:31:23Z,2022-01-01T14:57:35Z,5 Ave & E 87 St,7323.09,5 Ave & E 72 St,7100.07,40.782323,-73.959875,...,-73.966853,casual,13:31:23,2022-01-01,14:57:35,2022-01-01,Saturday,13,86.200000,0.672169
1010768,421BF85804C9D2C9,classic_bike,2022-01-01T13:35:42Z,2022-01-01T14:59:58Z,5 Ave & E 87 St,7323.09,5 Ave & E 72 St,7100.07,40.782323,-73.959875,...,-73.966853,casual,13:35:42,2022-01-01,14:59:58,2022-01-01,Saturday,13,84.266667,0.672169
1011423,1F936B6D7F7453FA,classic_bike,2022-01-04T21:57:32Z,2022-01-05T13:56:05Z,Broadway & Roebling St,5125.07,N 6 St & Bedford Ave,5379.1,40.709248,-73.960631,...,-73.958509,casual,21:57:32,2022-01-04,13:56:05,2022-01-05,Tuesday,21,958.550000,0.679189


In [19]:
trips_data[trips_data['anomaly_scores'] > 0.67]

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,end_lng,member_casual,start_time,start_date,end_time,end_date,day_of_week,hour,trip_duration,anomaly_scores
6451,71FC7480EA4EF5CF,classic_bike,2022-01-15T13:21:18Z,2022-01-15T14:25:30Z,W 42 St & Dyer Ave,6644.07,W 42 St & Dyer Ave,6644.07,40.758985,-73.993800,...,-73.993800,casual,13:21:18,2022-01-15,14:25:30,2022-01-15,Saturday,13,64.200000,0.672169
6615,C064DD3E2634226D,electric_bike,2022-01-19T23:47:53Z,2022-01-20T03:37:42Z,E 58 St & 1 Ave (NE Corner),6682.03,Old Fulton St,4903.08,40.758924,-73.962262,...,-73.993836,casual,23:47:53,2022-01-19,03:37:42,2022-01-20,Wednesday,23,229.816667,0.670750
17276,D334B9B7319F0150,classic_bike,2022-01-02T23:38:39Z,2022-01-03T00:27:36Z,Flatbush Ave & Eastern Pkwy,3856.03,Flatbush Ave & Ocean Ave,3704.04,40.668757,-73.966438,...,-73.963014,casual,23:38:39,2022-01-02,00:27:36,2022-01-03,Sunday,23,48.950000,0.670105
23619,E0378EEAA5C5E85E,classic_bike,2022-01-19T15:50:57Z,2022-01-19T17:08:26Z,Goulden Ave & W 197 St,8707.01,Goulden Ave & W 197 St,8707.01,40.871560,-73.897140,...,-73.897140,casual,15:50:57,2022-01-19,17:08:26,2022-01-19,Wednesday,15,77.483333,0.670673
32811,533FDE12BD24093E,classic_bike,2022-01-02T13:27:21Z,2022-01-02T14:33:26Z,W 48 St & Rockefeller Plaza,6626.11,7 Ave & Central Park South,6912.01,40.757769,-73.979294,...,-73.979069,casual,13:27:21,2022-01-02,14:33:26,2022-01-02,Sunday,13,66.083333,0.673960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1006242,BE3878D514C13453,classic_bike,2022-01-19T15:45:47Z,2022-01-19T16:58:32Z,Adam Clayton Powell Blvd & W 118 St,7670.09,5 Ave & W 126 St,7701.21,40.804372,-73.951475,...,-73.941800,casual,15:45:47,2022-01-19,16:58:32,2022-01-19,Wednesday,15,72.750000,0.670673
1007410,F8A3AD15AE1E11FF,classic_bike,2022-01-19T15:03:09Z,2022-01-19T16:11:40Z,5 Ave & E 63 St,6904.06,5 Ave & E 72 St,7100.07,40.766368,-73.971518,...,-73.966853,casual,15:03:09,2022-01-19,16:11:40,2022-01-19,Wednesday,15,68.516667,0.670673
1009464,04A4B63FA9BF2E01,classic_bike,2022-01-01T13:31:23Z,2022-01-01T14:57:35Z,5 Ave & E 87 St,7323.09,5 Ave & E 72 St,7100.07,40.782323,-73.959875,...,-73.966853,casual,13:31:23,2022-01-01,14:57:35,2022-01-01,Saturday,13,86.200000,0.672169
1010768,421BF85804C9D2C9,classic_bike,2022-01-01T13:35:42Z,2022-01-01T14:59:58Z,5 Ave & E 87 St,7323.09,5 Ave & E 72 St,7100.07,40.782323,-73.959875,...,-73.966853,casual,13:35:42,2022-01-01,14:59:58,2022-01-01,Saturday,13,84.266667,0.672169


In [20]:
trips_data[trips_data['anomaly_scores'] > 0.68]

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,end_lng,member_casual,start_time,start_date,end_time,end_date,day_of_week,hour,trip_duration,anomaly_scores
38199,A7D9ECFDD4EEF39C,classic_bike,2022-01-23T02:31:00Z,2022-01-23T03:39:15Z,Pearl St & Peck Slip,5096.11,Pearl St & Peck Slip,5096.11,40.708485,-74.002751,...,-74.002751,casual,02:31:00,2022-01-23,03:39:15,2022-01-23,Sunday,2,68.250000,0.685231
77285,D56CD0ED8ED5F338,classic_bike,2022-01-01T02:01:12Z,2022-01-01T03:06:08Z,Vernon Blvd & 50 Ave,6170.02,Henry St & Middagh St,4861.05,40.742327,-73.954117,...,-73.991581,casual,02:01:12,2022-01-01,03:06:08,2022-01-01,Saturday,2,64.933333,0.684235
105959,D081E4177C4BF1C3,electric_bike,2022-01-01T02:24:32Z,2022-01-01T03:19:21Z,MacDougal St & Washington Sq,5797.01,E 58 St & 3 Ave,6762.02,40.732264,-73.998522,...,-73.967245,casual,02:24:32,2022-01-01,03:19:21,2022-01-01,Saturday,2,54.816667,0.680394
149686,C70A70605640A040,electric_bike,2022-01-01T02:42:18Z,2022-01-01T03:58:20Z,10 Ave & W 207 St,8613.09,Morris Ave & E 171 St,8209.01,40.864632,-73.918598,...,-73.910900,casual,02:42:18,2022-01-01,03:58:20,2022-01-01,Saturday,2,76.033333,0.681777
159764,2FD7D23A3B6C0469,electric_bike,2022-01-31T02:11:09Z,2022-01-31T03:05:14Z,E 110 St & Madison Ave,7587.14,8 Ave & W 38 St,6526.05,40.796154,-73.947821,...,-73.991770,casual,02:11:09,2022-01-31,03:05:14,2022-01-31,Monday,2,54.083333,0.684569
175034,6852290F2FB4020E,classic_bike,2022-01-23T02:36:36Z,2022-01-23T03:39:39Z,Pearl St & Peck Slip,5096.11,Pearl St & Peck Slip,5096.11,40.708485,-74.002751,...,-74.002751,casual,02:36:36,2022-01-23,03:39:39,2022-01-23,Sunday,2,63.050000,0.685231
176706,88D2478DC86A1FC7,classic_bike,2022-01-23T02:35:27Z,2022-01-23T03:39:26Z,Pearl St & Peck Slip,5096.11,Pearl St & Peck Slip,5096.11,40.708485,-74.002751,...,-74.002751,casual,02:35:27,2022-01-23,03:39:26,2022-01-23,Sunday,2,63.983333,0.685231
176711,0224ACDAF5737DA2,classic_bike,2022-01-23T02:29:21Z,2022-01-23T03:38:52Z,Pearl St & Peck Slip,5096.11,Pearl St & Peck Slip,5096.11,40.708485,-74.002751,...,-74.002751,casual,02:29:21,2022-01-23,03:38:52,2022-01-23,Sunday,2,69.516667,0.685231
233217,177285825E8966E9,electric_bike,2022-01-01T02:43:09Z,2022-01-01T09:03:02Z,21 St & Queens Plaza North,6471.02,E 81 St & 2 Ave,7121.05,40.753260,-73.943358,...,-73.954275,casual,02:43:09,2022-01-01,09:03:02,2022-01-01,Saturday,2,379.883333,0.681777
252974,5D28D29F6857B81E,electric_bike,2022-01-13T02:21:15Z,2022-01-13T13:37:01Z,Lenox Ave & W 117 St,7655.22,2 Ave & E 96 St,7338.02,40.802557,-73.949078,...,-73.947167,casual,02:21:15,2022-01-13,13:37:01,2022-01-13,Thursday,2,675.766667,0.681339


In [95]:
top_10_routes_trips=  pd.read_csv('G:/NYCCitiBikeSystem_AnomalyDetection/data/top_10_routes_trips.csv')
top_10_routes_trips.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,start_time,start_date,end_time,end_date,day_of_week,hour,trip_duration
0,C04D1CDBA0C731EA,classic_bike,2022-01-19T10:26:16Z,2022-01-19T10:31:39Z,W 21 St & 6 Ave,6140.05,W 22 St & 10 Ave,6306.06,40.74174,-73.994156,40.74692,-74.004519,member,10:26:16,2022-01-19,10:31:39,2022-01-19,Wednesday,10,5.383333
1,BEB356FD0A291DB9,classic_bike,2022-01-08T18:04:49Z,2022-01-08T18:11:45Z,W 21 St & 6 Ave,6140.05,W 22 St & 10 Ave,6306.06,40.74174,-73.994156,40.74692,-74.004519,member,18:04:49,2022-01-08,18:11:45,2022-01-08,Saturday,18,6.933333
2,3517E905BE19D91F,classic_bike,2022-01-31T09:02:40Z,2022-01-31T09:11:44Z,W 21 St & 6 Ave,6140.05,W 22 St & 10 Ave,6306.06,40.74174,-73.994156,40.74692,-74.004519,member,09:02:40,2022-01-31,09:11:44,2022-01-31,Monday,9,9.066667
3,40FC1E5CD0267EA8,electric_bike,2022-01-18T09:47:04Z,2022-01-18T09:51:33Z,W 21 St & 6 Ave,6140.05,W 22 St & 10 Ave,6306.06,40.74174,-73.994156,40.74692,-74.004519,member,09:47:04,2022-01-18,09:51:33,2022-01-18,Tuesday,9,4.483333
4,B8E69F9E7DB12097,electric_bike,2022-01-02T10:11:03Z,2022-01-02T10:15:04Z,W 21 St & 6 Ave,6140.05,W 22 St & 10 Ave,6306.06,40.74174,-73.994156,40.74692,-74.004519,member,10:11:03,2022-01-02,10:15:04,2022-01-02,Sunday,10,4.016667


In [96]:
top_10_routes_trips['rideable_type'] = top_10_routes_trips['rideable_type'].astype('category')
top_10_routes_trips['member_casual'] = top_10_routes_trips['member_casual'].astype('category')
top_10_routes_trips['day_of_week'] = top_10_routes_trips['day_of_week'].astype('category')
top_10_routes_trips['start_station_id'] = top_10_routes_trips['start_station_id'].astype('category')
top_10_routes_trips['end_station_id'] = top_10_routes_trips['end_station_id'].astype('category')
top_10_routes_trips['hour'] = top_10_routes_trips['hour'].astype('category')

In [97]:
# select variables
features = ['rideable_type','start_station_id','end_station_id','member_casual','day_of_week','trip_duration','hour']


training_data = top_10_routes_trips[features]

   
model_1 = IsolationForest(n_estimators=100, max_samples=256, bootstrap = True, n_jobs=11)
model_1.fit(training_data)


In [98]:
top_10_routes_trips['anomaly_scores'] = model_1.score_samples(training_data)

In [99]:
# sort by anomaly scores
top_10_routes_trips.sort_values(by='anomaly_scores', ascending=False)


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,end_lng,member_casual,start_time,start_date,end_time,end_date,day_of_week,hour,trip_duration,anomaly_scores
1561,0569F2945599831E,classic_bike,2022-01-10T09:34:47Z,2022-01-10T16:43:11Z,46 Ave & 5 St,6286.02,Vernon Blvd & 50 Ave,6170.02,40.747310,-73.954510,...,-73.954117,member,09:34:47,2022-01-10,16:43:11,2022-01-10,Monday,9,428.400000,0.614792
736,AA4EB856C882BF78,electric_bike,2022-01-16T16:01:42Z,2022-01-17T14:10:52Z,Central Park S & 6 Ave,6876.04,Central Park S & 6 Ave,6876.04,40.765909,-73.976342,...,-73.976342,casual,16:01:42,2022-01-16,14:10:52,2022-01-17,Sunday,16,1329.166667,0.614667
722,4BD978D8249D0BD6,electric_bike,2022-01-25T14:46:47Z,2022-01-25T16:08:17Z,Central Park S & 6 Ave,6876.04,Central Park S & 6 Ave,6876.04,40.765909,-73.976342,...,-73.976342,casual,14:46:47,2022-01-25,16:08:17,2022-01-25,Tuesday,14,81.500000,0.602257
790,8D5369123EB697DC,classic_bike,2022-01-19T10:00:48Z,2022-01-20T00:04:31Z,Central Park S & 6 Ave,6876.04,Central Park S & 6 Ave,6876.04,40.765909,-73.976342,...,-73.976342,casual,10:00:48,2022-01-19,00:04:31,2022-01-20,Wednesday,10,843.716667,0.593900
652,862EA29FD341E435,electric_bike,2022-01-02T14:55:37Z,2022-01-02T16:39:36Z,Central Park S & 6 Ave,6876.04,Central Park S & 6 Ave,6876.04,40.765909,-73.976342,...,-73.976342,casual,14:55:37,2022-01-02,16:39:36,2022-01-02,Sunday,14,103.983333,0.587870
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2429,C511162509AD370F,classic_bike,2022-01-11T18:01:06Z,2022-01-11T18:05:13Z,W 21 St & 6 Ave,6140.05,9 Ave & W 22 St,6266.06,40.741740,-73.994156,...,-74.001971,member,18:01:06,2022-01-11,18:05:13,2022-01-11,Tuesday,18,4.116667,0.374810
2355,1C5703A46EDD4141,classic_bike,2022-01-25T17:31:16Z,2022-01-25T17:36:36Z,W 21 St & 6 Ave,6140.05,9 Ave & W 22 St,6266.06,40.741740,-73.994156,...,-74.001971,member,17:31:16,2022-01-25,17:36:36,2022-01-25,Tuesday,17,5.333333,0.374700
2217,AC6ECDC5BEDE754E,classic_bike,2022-01-25T17:40:34Z,2022-01-25T17:45:40Z,W 21 St & 6 Ave,6140.05,9 Ave & W 22 St,6266.06,40.741740,-73.994156,...,-74.001971,member,17:40:34,2022-01-25,17:45:40,2022-01-25,Tuesday,17,5.100000,0.374700
2197,9C92EA071510A615,classic_bike,2022-01-18T19:54:31Z,2022-01-18T19:58:25Z,W 21 St & 6 Ave,6140.05,9 Ave & W 22 St,6266.06,40.741740,-73.994156,...,-74.001971,member,19:54:31,2022-01-18,19:58:25,2022-01-18,Tuesday,19,3.900000,0.374621


In [100]:
top_10_routes_trips[top_10_routes_trips['anomaly_scores'] > 0.5]

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,end_lng,member_casual,start_time,start_date,end_time,end_date,day_of_week,hour,trip_duration,anomaly_scores
25,2E47B8C7A63E9045,electric_bike,2022-01-06T17:44:30Z,2022-01-06T18:06:36Z,W 21 St & 6 Ave,6140.05,W 22 St & 10 Ave,6306.06,40.741740,-73.994156,...,-74.004519,member,17:44:30,2022-01-06,18:06:36,2022-01-06,Thursday,17,22.100000,0.500513
135,E652BAFE4384DAE5,electric_bike,2022-01-16T14:41:13Z,2022-01-16T15:15:16Z,W 21 St & 6 Ave,6140.05,W 22 St & 10 Ave,6306.06,40.741740,-73.994156,...,-74.004519,member,14:41:13,2022-01-16,15:15:16,2022-01-16,Sunday,14,34.050000,0.519397
141,6E1A51A2EEF31E71,electric_bike,2022-01-12T08:50:43Z,2022-01-12T08:59:30Z,W 21 St & 6 Ave,6140.05,W 22 St & 10 Ave,6306.06,40.741740,-73.994156,...,-74.004519,casual,08:50:43,2022-01-12,08:59:30,2022-01-12,Wednesday,8,8.783333,0.508689
181,0DD0F34A282DB71F,electric_bike,2022-01-06T13:05:16Z,2022-01-06T13:14:24Z,W 21 St & 6 Ave,6140.05,W 22 St & 10 Ave,6306.06,40.741740,-73.994156,...,-74.004519,casual,13:05:16,2022-01-06,13:14:24,2022-01-06,Thursday,13,9.133333,0.511950
194,57152D28D81EAD6F,electric_bike,2022-01-17T16:23:08Z,2022-01-17T16:30:34Z,W 21 St & 6 Ave,6140.05,W 22 St & 10 Ave,6306.06,40.741740,-73.994156,...,-74.004519,casual,16:23:08,2022-01-17,16:30:34,2022-01-17,Monday,16,7.433333,0.515894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,22C78C263F26EDAD,electric_bike,2022-01-03T09:05:03Z,2022-01-03T09:09:05Z,West St & Chambers St,5329.03,Pier 40 - Hudson River Park,5696.03,40.717548,-74.013221,...,-74.011296,casual,09:05:03,2022-01-03,09:09:05,2022-01-03,Monday,9,4.033333,0.500884
2162,F291F83550109B35,classic_bike,2022-01-09T10:32:53Z,2022-01-09T13:27:49Z,West St & Chambers St,5329.03,Pier 40 - Hudson River Park,5696.03,40.717548,-74.013221,...,-74.011296,member,10:32:53,2022-01-09,13:27:49,2022-01-09,Sunday,10,174.933333,0.517971
2493,471CC2C96142C730,classic_bike,2022-01-01T19:10:37Z,2022-01-01T20:29:54Z,W 21 St & 6 Ave,6140.05,9 Ave & W 22 St,6266.06,40.741740,-73.994156,...,-74.001971,casual,19:10:37,2022-01-01,20:29:54,2022-01-01,Saturday,19,79.283333,0.568749
2494,39305C9DF9F31E25,classic_bike,2022-01-01T19:09:33Z,2022-01-01T20:31:29Z,W 21 St & 6 Ave,6140.05,9 Ave & W 22 St,6266.06,40.741740,-73.994156,...,-74.001971,casual,19:09:33,2022-01-01,20:31:29,2022-01-01,Saturday,19,81.933333,0.570030


In [105]:
top_10_routes_trips[top_10_routes_trips['anomaly_scores'] > 0.55].sort_values(by='anomaly_scores', ascending=False)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,end_lng,member_casual,start_time,start_date,end_time,end_date,day_of_week,hour,trip_duration,anomaly_scores
1561,0569F2945599831E,classic_bike,2022-01-10T09:34:47Z,2022-01-10T16:43:11Z,46 Ave & 5 St,6286.02,Vernon Blvd & 50 Ave,6170.02,40.747310,-73.954510,...,-73.954117,member,09:34:47,2022-01-10,16:43:11,2022-01-10,Monday,9,428.400000,0.614792
736,AA4EB856C882BF78,electric_bike,2022-01-16T16:01:42Z,2022-01-17T14:10:52Z,Central Park S & 6 Ave,6876.04,Central Park S & 6 Ave,6876.04,40.765909,-73.976342,...,-73.976342,casual,16:01:42,2022-01-16,14:10:52,2022-01-17,Sunday,16,1329.166667,0.614667
722,4BD978D8249D0BD6,electric_bike,2022-01-25T14:46:47Z,2022-01-25T16:08:17Z,Central Park S & 6 Ave,6876.04,Central Park S & 6 Ave,6876.04,40.765909,-73.976342,...,-73.976342,casual,14:46:47,2022-01-25,16:08:17,2022-01-25,Tuesday,14,81.500000,0.602257
790,8D5369123EB697DC,classic_bike,2022-01-19T10:00:48Z,2022-01-20T00:04:31Z,Central Park S & 6 Ave,6876.04,Central Park S & 6 Ave,6876.04,40.765909,-73.976342,...,-73.976342,casual,10:00:48,2022-01-19,00:04:31,2022-01-20,Wednesday,10,843.716667,0.593900
652,862EA29FD341E435,electric_bike,2022-01-02T14:55:37Z,2022-01-02T16:39:36Z,Central Park S & 6 Ave,6876.04,Central Park S & 6 Ave,6876.04,40.765909,-73.976342,...,-73.976342,casual,14:55:37,2022-01-02,16:39:36,2022-01-02,Sunday,14,103.983333,0.587870
1996,97CC0452290B5C10,electric_bike,2022-01-19T22:24:49Z,2022-01-19T23:08:08Z,West St & Chambers St,5329.03,Pier 40 - Hudson River Park,5696.03,40.717548,-74.013221,...,-74.011296,casual,22:24:49,2022-01-19,23:08:08,2022-01-19,Wednesday,22,43.316667,0.584970
481,3B6C5C2A202DDD87,electric_bike,2022-01-14T20:25:27Z,2022-01-14T20:46:39Z,1 Ave & E 62 St,6753.08,1 Ave & E 68 St,6822.09,40.761227,-73.960940,...,-73.958185,casual,20:25:27,2022-01-14,20:46:39,2022-01-14,Friday,20,21.200000,0.584862
804,2770BF64E63E12F0,electric_bike,2022-01-19T10:03:59Z,2022-01-19T11:21:20Z,Central Park S & 6 Ave,6876.04,Central Park S & 6 Ave,6876.04,40.765909,-73.976342,...,-73.976342,casual,10:03:59,2022-01-19,11:21:20,2022-01-19,Wednesday,10,77.350000,0.582352
1414,FE009FAC88E49C68,electric_bike,2022-01-29T01:23:41Z,2022-01-29T01:35:28Z,1 Ave & E 62 St,6753.08,1 Ave & E 68 St,6822.09,40.761227,-73.960940,...,-73.958185,casual,01:23:41,2022-01-29,01:35:28,2022-01-29,Saturday,1,11.783333,0.579937
700,4954FD092A4A5495,electric_bike,2022-01-13T14:02:08Z,2022-01-13T14:56:07Z,Central Park S & 6 Ave,6876.04,Central Park S & 6 Ave,6876.04,40.765909,-73.976342,...,-73.976342,casual,14:02:08,2022-01-13,14:56:07,2022-01-13,Thursday,14,53.983333,0.577490


In [102]:
top_10_routes_trips[top_10_routes_trips['anomaly_scores'] > 0.6]

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,end_lng,member_casual,start_time,start_date,end_time,end_date,day_of_week,hour,trip_duration,anomaly_scores
722,4BD978D8249D0BD6,electric_bike,2022-01-25T14:46:47Z,2022-01-25T16:08:17Z,Central Park S & 6 Ave,6876.04,Central Park S & 6 Ave,6876.04,40.765909,-73.976342,...,-73.976342,casual,14:46:47,2022-01-25,16:08:17,2022-01-25,Tuesday,14,81.500000,0.602257
736,AA4EB856C882BF78,electric_bike,2022-01-16T16:01:42Z,2022-01-17T14:10:52Z,Central Park S & 6 Ave,6876.04,Central Park S & 6 Ave,6876.04,40.765909,-73.976342,...,-73.976342,casual,16:01:42,2022-01-16,14:10:52,2022-01-17,Sunday,16,1329.166667,0.614667
1561,0569F2945599831E,classic_bike,2022-01-10T09:34:47Z,2022-01-10T16:43:11Z,46 Ave & 5 St,6286.02,Vernon Blvd & 50 Ave,6170.02,40.747310,-73.954510,...,-73.954117,member,09:34:47,2022-01-10,16:43:11,2022-01-10,Monday,9,428.400000,0.614792
